In [118]:
import pandas as pd
import numpy as np
%matplotlib inline

In [60]:
!ls -l input

total 2451736
-rw-r--r--@ 1 alexandrelacheze  staff  1037267659 Jul  8 01:23 app_events.csv
-rw-r--r--@ 1 alexandrelacheze  staff    11190003 Jul  8 01:28 app_labels.csv
-rw-r--r--@ 1 alexandrelacheze  staff   195433779 Jul  8 01:22 events.csv
-rw-r--r--@ 1 alexandrelacheze  staff     2284333 Jul  8 01:28 gender_age_test.csv
-rw-r--r--@ 1 alexandrelacheze  staff     2366486 Jul 11 23:23 gender_age_train.csv
-rw-r--r--@ 1 alexandrelacheze  staff       16450 Jul  8 01:28 label_categories.csv
-rw-r--r--@ 1 alexandrelacheze  staff     6715635 Jul  8 01:28 phone_brand_device_model.csv


In [61]:
# limitation
NROWS = 10000

In [62]:
train = pd.read_csv("./input/gender_age_train.csv")
test =  pd.read_csv("./input/gender_age_test.csv")

In [63]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
device_id    74645 non-null int64
gender       74645 non-null object
age          74645 non-null int64
group        74645 non-null object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [64]:
phone_brand_device_model =  pd.read_csv("./input/phone_brand_device_model.csv")

In [65]:
phone_brand_device_model.head()

,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4
3,3669464369358936369,SUGAR,时尚手机
4,-5019277647504317457,三星,Galaxy Note 2


In [66]:
train = pd.merge(train, phone_brand_device_model, how='left')
test = pd.merge(test, phone_brand_device_model, how='left')

In [67]:
train.head()

,device_id,gender,age,group,phone_brand,device_model
0,-8076087639492063270,M,35,M32-38,小米,MI 2
1,-2897161552818060146,M,35,M32-38,小米,MI 2
2,-8260683887967679142,M,35,M32-38,小米,MI 2
3,-4938849341048082022,M,30,M29-31,小米,红米note
4,245133531816851882,M,30,M29-31,小米,MI 3


In [68]:
from sklearn import cross_validation
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [69]:
le = preprocessing.LabelEncoder()
le.fit(phone_brand_device_model['phone_brand'].tolist())
train['phone_brand_enc'] = le.transform(train['phone_brand'])
test['phone_brand_enc'] = le.transform(test['phone_brand'])

le = preprocessing.LabelEncoder()
le.fit(phone_brand_device_model['device_model'].tolist())
train['device_model_enc'] = le.transform(train['device_model'])
test['device_model_enc'] = le.transform(test['device_model'])

In [70]:
X = train[['phone_brand_enc', 'device_model_enc']]
y = train['group']

In [71]:
#clf = svm.LinearSVC()
clf = RandomForestClassifier()
scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='log_loss')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: -4.16 (+/- 0.31)


In [72]:
clf.fit(X, y)
results=pd.DataFrame(clf.predict_proba(test[['phone_brand_enc', 'device_model_enc']]), columns=clf.classes_)
results['device_id'] = test['device_id']

In [87]:
results.drop_duplicates('device_id', inplace=True)
results.to_csv('./output/submission.csv', index=False)

In [75]:
app_ev = pd.read_csv("./input/app_events.csv",
                     dtype={'device_id': np.str},
                     nrows=NROWS
                    )

In [76]:
app_ev.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


In [77]:
events = pd.read_csv("./input/events.csv",
                    dtype={'device_id': np.str},
                    nrows=NROWS
                    )

In [78]:
events

,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66
5,6,1476664663289716375,2016-05-01 00:27:21,0.00,0.00
6,7,5990807147117726237,2016-05-01 00:15:13,113.73,23.00
7,8,1782450055857303792,2016-05-01 00:15:35,113.94,34.70
8,9,-2073340001552902943,2016-05-01 00:15:33,0.00,0.00
9,10,-8195816569128397698,2016-05-01 00:41:31,119.34,26.04


## Handle geolocation now

In [128]:
from pygeocoder import Geocoder
from geopandas import GeoDataFrame
import geopandas as gpd
from shapely.geometry import Point

In [98]:
results = Geocoder.reverse_geocode(events.iloc[1]["latitude"], events.iloc[1]["longitude"])

In [102]:
results.data

[{u'address_components': [{u'long_name': u'543\u53f7-615\u53f7',
    u'short_name': u'543\u53f7-615\u53f7',
    u'types': [u'street_number']},
   {u'long_name': u'Tong Jin Lu',
    u'short_name': u'Tong Jin Lu',
    u'types': [u'route']},
   {u'long_name': u'Dujiangyan Shi',
    u'short_name': u'Dujiangyan Shi',
    u'types': [u'political', u'sublocality', u'sublocality_level_1']},
   {u'long_name': u'Chengdu Shi',
    u'short_name': u'Chengdu Shi',
    u'types': [u'locality', u'political']},
   {u'long_name': u'Sichuan Sheng',
    u'short_name': u'Sichuan Sheng',
    u'types': [u'administrative_area_level_1', u'political']},
   {u'long_name': u'China',
    u'short_name': u'CN',
    u'types': [u'country', u'political']},
   {u'long_name': u'611830',
    u'short_name': u'611830',
    u'types': [u'postal_code']}],
  u'formatted_address': u'543\u53f7-615 Tong Jin Lu, Dujiangyan Shi, Chengdu Shi, Sichuan Sheng, China, 611830',
  u'geometry': {u'bounds': {u'northeast': {u'lat': 30.9703897,


In [104]:
results.locality

u'Chengdu Shi'

In [115]:
events['lat_cleaned'] = events['latitude'].replace((0.00, 1.00), pd.np.NaN)
events['lon_cleaned'] = events['longitude'].replace((0.00, 1.00), pd.np.NaN)
events['geometry'] = events.apply(lambda row: Point((row['lat_cleaned'], row['lon_cleaned'])), axis=1)
gevents =  GeoDataFrame(events)

In [129]:
cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [131]:
cities.head()

,geometry,name
0,POINT (12.45338654497177 41.90328217996012),Vatican City
1,POINT (12.44177015780014 43.936095834768),San Marino
2,POINT (9.516669472907267 47.13372377429357),Vaduz
3,POINT (6.130002806227083 49.61166037912108),Luxembourg
4,POINT (158.1499743237623 6.916643696007725),Palikir
